<a href="https://github.com/timeseriesAI/tsai-rs" target="_parent"><img src="https://img.shields.io/badge/tsai--rs-Time%20Series%20AI%20in%20Rust-blue" alt="tsai-rs"/></a>

# Self-Supervised Learning for Time Series with tsai-rs

This notebook demonstrates self-supervised pretraining concepts for time series using **tsai-rs**.

## Introduction

Self-supervised learning allows models to learn useful representations from unlabeled data. This is particularly valuable for time series when:

1. **Limited labeled data**: Only a small fraction of data has labels
2. **Expensive labeling**: Getting labels is costly or time-consuming
3. **Transfer learning**: Pre-train on one domain, fine-tune on another

Common self-supervised approaches for time series:
- **MVP/TSBERT**: Mask-based pretraining (similar to BERT)
- **Contrastive learning**: Learn to distinguish similar/dissimilar pairs
- **Autoencoding**: Reconstruct input time series

## Install tsai-rs

```bash
cd crates/tsai_python
maturin develop --release
```

## Import Libraries

In [ ]:
import tsai_rs
import numpy as np
import matplotlib.pyplot as plt

print(f"tsai-rs version: {tsai_rs.version()}")
tsai_rs.my_setup()

## Load Data

In [ ]:
# Load multivariate dataset
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

n_vars = X_train.shape[1]
seq_len = X_train.shape[2]
n_classes = len(np.unique(y_train))

print(f"Dataset: {dsid}")
print(f"X_train shape: {X_train.shape}")
print(f"Classes: {n_classes}")

In [ ]:
# Standardize
X_train_std = tsai_rs.ts_standardize(X_train.astype(np.float32), by_sample=True)
X_test_std = tsai_rs.ts_standardize(X_test.astype(np.float32), by_sample=True)

## Data Augmentation for Self-Supervised Learning

Self-supervised learning often relies on data augmentation to create different "views" of the same sample.

In [ ]:
def create_augmented_view(X, seed=None):
    """Create an augmented view of the time series data."""
    X_aug = X.copy()
    
    # Apply random augmentations
    X_aug = tsai_rs.add_gaussian_noise(X_aug, std=0.05, seed=seed)
    X_aug = tsai_rs.mag_scale(X_aug, scale_range=(0.9, 1.1), seed=seed + 100 if seed else None)
    
    return X_aug

# Create two views of the same sample
sample = X_train_std[:1]
view1 = create_augmented_view(sample, seed=42)
view2 = create_augmented_view(sample, seed=123)

# Visualize
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

axes[0].plot(sample[0, 0, :])
axes[0].set_title('Original')

axes[1].plot(view1[0, 0, :])
axes[1].set_title('Augmented View 1')

axes[2].plot(view2[0, 0, :])
axes[2].set_title('Augmented View 2')

plt.tight_layout()
plt.show()

## Masking for Self-Supervised Learning

Similar to BERT, we can mask portions of the time series and train a model to reconstruct them.

In [ ]:
def create_masked_input(X, mask_ratio=0.15, seed=None):
    """Create masked version of time series.
    
    Args:
        X: Input data (samples, vars, length)
        mask_ratio: Proportion of timesteps to mask
        seed: Random seed
    
    Returns:
        X_masked: Masked input
        mask: Boolean mask (True = masked)
    """
    if seed is not None:
        np.random.seed(seed)
    
    n_samples, n_vars, seq_len = X.shape
    n_mask = int(seq_len * mask_ratio)
    
    X_masked = X.copy()
    masks = np.zeros((n_samples, seq_len), dtype=bool)
    
    for i in range(n_samples):
        # Select random positions to mask
        mask_positions = np.random.choice(seq_len, n_mask, replace=False)
        masks[i, mask_positions] = True
        
        # Set masked positions to 0 (or could use special token)
        X_masked[i, :, mask_positions] = 0
    
    return X_masked, masks

# Create masked version
X_masked, masks = create_masked_input(X_train_std[:5], mask_ratio=0.2, seed=42)

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

axes[0].plot(X_train_std[0, 0, :])
axes[0].set_title('Original')

axes[1].plot(X_masked[0, 0, :])
# Mark masked positions
masked_pos = np.where(masks[0])[0]
axes[1].scatter(masked_pos, X_masked[0, 0, masked_pos], c='red', s=50, zorder=5, label='Masked')
axes[1].set_title(f'Masked (20% = {len(masked_pos)} timesteps)')
axes[1].legend()

plt.tight_layout()
plt.show()

## Model Configuration for Self-Supervised Learning

Models used in self-supervised pretraining typically have:
1. An encoder that learns representations
2. A projection head for the pretraining task
3. After pretraining, replace projection head with classification/regression head

In [ ]:
# Encoder configuration (for pretraining)
# Use same architecture but with reconstruction head
encoder_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=seq_len * n_vars  # Reconstruct all timesteps
)
print(f"Encoder config (for reconstruction): {encoder_config}")

In [ ]:
# After pretraining: use smaller head for classification
classifier_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes  # Number of classes
)
print(f"Classifier config: {classifier_config}")

## Simulating Limited Labels Scenario

In [ ]:
def create_limited_label_splits(y, label_fraction=0.1, seed=42):
    """Create train/valid splits with limited labels.
    
    Args:
        y: Labels
        label_fraction: Fraction of labeled training data
        seed: Random seed
    
    Returns:
        labeled_idx: Indices of labeled samples
        unlabeled_idx: Indices of unlabeled samples
    """
    np.random.seed(seed)
    
    n_samples = len(y)
    n_labeled = int(n_samples * label_fraction)
    
    # Stratified sampling to maintain class distribution
    labeled_idx = []
    classes = np.unique(y)
    n_per_class = max(1, n_labeled // len(classes))
    
    for cls in classes:
        cls_idx = np.where(y == cls)[0]
        selected = np.random.choice(cls_idx, min(n_per_class, len(cls_idx)), replace=False)
        labeled_idx.extend(selected)
    
    labeled_idx = np.array(labeled_idx)
    unlabeled_idx = np.setdiff1d(np.arange(n_samples), labeled_idx)
    
    return labeled_idx, unlabeled_idx

# Create 10% labeled split
labeled_idx, unlabeled_idx = create_limited_label_splits(y_train, label_fraction=0.1)

print(f"Total training samples: {len(y_train)}")
print(f"Labeled samples (10%): {len(labeled_idx)}")
print(f"Unlabeled samples: {len(unlabeled_idx)}")

# Class distribution in labeled subset
classes, counts = np.unique(y_train[labeled_idx], return_counts=True)
print(f"\nClass distribution in labeled subset:")
for c, cnt in zip(classes, counts):
    print(f"  Class {c}: {cnt} samples")

## Self-Supervised Training Pipeline

In [ ]:
def ssl_training_pipeline(X_train, y_train, X_test, y_test, label_fraction=0.1, seed=42):
    """Simulate self-supervised learning pipeline.
    
    Steps:
    1. Split data into labeled/unlabeled
    2. Pretrain on ALL data (no labels needed)
    3. Fine-tune on labeled subset
    """
    # Step 1: Create splits
    labeled_idx, unlabeled_idx = create_limited_label_splits(y_train, label_fraction, seed)
    
    # Labeled and unlabeled data
    X_labeled = X_train[labeled_idx]
    y_labeled = y_train[labeled_idx]
    X_unlabeled = X_train[unlabeled_idx]
    
    print(f"Step 1: Data Split")
    print(f"  Labeled samples: {len(X_labeled)}")
    print(f"  Unlabeled samples: {len(X_unlabeled)}")
    
    # Step 2: Pretrain on ALL data (both labeled and unlabeled)
    print(f"\nStep 2: Pretraining on ALL {len(X_train)} samples")
    print(f"  (Uses no labels - self-supervised)")
    
    # During pretraining, would:
    # - Mask random portions of time series
    # - Train model to reconstruct masked portions
    # - Or use contrastive learning with augmented views
    
    # Step 3: Fine-tune on labeled data only
    print(f"\nStep 3: Fine-tuning on {len(X_labeled)} labeled samples")
    print(f"  (Uses {label_fraction*100:.0f}% of labels)")
    
    return labeled_idx, unlabeled_idx

# Run pipeline simulation
labeled_idx, unlabeled_idx = ssl_training_pipeline(
    X_train_std, y_train, X_test_std, y_test,
    label_fraction=0.1
)

## Comparing Supervised vs Self-Supervised

In [ ]:
# Create configurations for different label fractions
label_fractions = [0.1, 0.25, 0.5, 1.0]

print(f"{'Label %':<10} {'Labeled':<10} {'Unlabeled':<12} {'SSL Benefit'}")
print("-" * 50)

for frac in label_fractions:
    labeled_idx, unlabeled_idx = create_limited_label_splits(y_train, frac)
    n_labeled = len(labeled_idx)
    n_unlabeled = len(unlabeled_idx)
    
    # SSL benefits more when labeled data is scarce
    benefit = "High" if frac <= 0.1 else "Medium" if frac <= 0.5 else "Lower"
    
    print(f"{frac*100:>6.0f}%    {n_labeled:<10} {n_unlabeled:<12} {benefit}")

## Creating Datasets for SSL

In [ ]:
# Unlabeled dataset (for pretraining)
# Note: y is not used during pretraining
X_pretrain = X_train_std  # All training data
print(f"Pretraining dataset: {X_pretrain.shape}")

# Labeled dataset (for fine-tuning)
labeled_idx, _ = create_limited_label_splits(y_train, label_fraction=0.1)
X_finetune = X_train_std[labeled_idx]
y_finetune = y_train[labeled_idx]

train_ds = tsai_rs.TSDataset(X_finetune, y_finetune)
print(f"Fine-tuning dataset: {train_ds}")

## Model Configuration

In [ ]:
# Configuration for different training scenarios

# Pretraining: Larger model, more capacity
pretrain_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    nf=64,  # More filters
    depth=8  # Deeper network
)

# Fine-tuning: Use pretrained weights
finetune_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    nf=64,
    depth=8,
    fc_dropout=0.3  # Add dropout for fine-tuning
)

print(f"Pretrain config: {pretrain_config}")
print(f"Fine-tune config: {finetune_config}")

## Training Schedule for SSL

In [ ]:
# Pretraining: Longer training, lower LR at end
pretrain_epochs = 200
pretrain_lr = 1e-2

# Fine-tuning: Shorter training, lower LR
finetune_epochs = 50
finetune_lr = 1e-3  # Lower LR for fine-tuning

# Calculate steps
batch_size = 32
n_pretrain = len(X_train_std)
n_finetune = len(X_finetune)

pretrain_steps = pretrain_epochs * ((n_pretrain + batch_size - 1) // batch_size)
finetune_steps = finetune_epochs * ((n_finetune + batch_size - 1) // batch_size)

pretrain_scheduler = tsai_rs.OneCycleLR.simple(max_lr=pretrain_lr, total_steps=pretrain_steps)
finetune_scheduler = tsai_rs.OneCycleLR.simple(max_lr=finetune_lr, total_steps=finetune_steps)

print("Self-Supervised Learning Schedule:")
print("=" * 40)
print(f"\nPretraining:")
print(f"  Epochs: {pretrain_epochs}")
print(f"  Samples: {n_pretrain} (all data, no labels)")
print(f"  LR: {pretrain_lr}")
print(f"  Total steps: {pretrain_steps}")

print(f"\nFine-tuning:")
print(f"  Epochs: {finetune_epochs}")
print(f"  Samples: {n_finetune} (10% labeled)")
print(f"  LR: {finetune_lr}")
print(f"  Total steps: {finetune_steps}")

## Summary

This notebook demonstrated self-supervised learning concepts for time series:

### Key Concepts
1. **Data Augmentation**: Create different views of the same sample
2. **Masking**: Hide parts of the time series for reconstruction
3. **Limited Labels**: Benefit most when labels are scarce

### Self-Supervised Pipeline
1. **Pretrain** on ALL data (no labels)
2. **Fine-tune** on labeled subset

### Benefits
- Better performance with limited labels
- Learns useful representations from unlabeled data
- Can leverage large unlabeled datasets

### tsai-rs Functions Used
- `ts_standardize`: Normalize data
- `add_gaussian_noise`: Data augmentation
- `mag_scale`: Magnitude scaling augmentation
- Model configs: `InceptionTimePlusConfig`, etc.
- Training: `LearnerConfig`, `OneCycleLR`

In [ ]:
# Quick reference for SSL with tsai-rs
print("Self-Supervised Learning with tsai-rs:")
print("=" * 50)
print("\n1. Load and standardize data:")
print("   X_train, y_train, _, _ = tsai_rs.get_UCR_data(dsid)")
print("   X_std = tsai_rs.ts_standardize(X_train, by_sample=True)")

print("\n2. Create augmented views:")
print("   X_aug = tsai_rs.add_gaussian_noise(X_std, std=0.05)")
print("   X_aug = tsai_rs.mag_scale(X_aug, scale_range=(0.9, 1.1))")

print("\n3. Configure model:")
print("   config = tsai_rs.InceptionTimePlusConfig(...)")

print("\n4. Pretrain (no labels):")
print("   # Train with reconstruction/contrastive loss")
print("   # Use all data")

print("\n5. Fine-tune (with labels):")
print("   # Load pretrained weights")
print("   # Train classifier head on labeled data")